In [2]:
import pandas as pd

import spotify
import utils

sp = spotify.login(scope='playlist-modify-private')
user = sp.current_user()
displayName = user['display_name']
print(displayName)

Enter the URL you were redirected to:  https://localhost:8080/?code=AQBeXsDZYJiM12RneHAxfPSnljgz5Nryp57R4G9pvmfuc_nm6Q4-OOcqimgCx_M1GQuwnV3otAO93gO5QUGJIUohFeYEjm0Y5T0b7-xnOsRz362QhMcyQpME2SPdBpl1ssg1NA-xeOdVX__d3VN0pSV_6dbqb298XiXr_F2HI2ogtAUDULHW2f1Ku2TUxhpOl5aqUA


mbirgi


In [3]:
# get all categories 
results = sp.categories(country='CH')['categories']
categories = results['items']
while results['next']:
    results = sp.next(results)['categories']
    categories.extend(results['items'])
print(f"{len(categories)} categories found")

62 categories found


In [4]:
[(ctg['id'], ctg['name']) for ctg in categories]

[('0JQ5DAqbMKFC7do0jUgBzi', '2024 in Music'),
 ('0JQ5DAt0tbjZptfcdMSKl3', 'Made For You'),
 ('0JQ5DAqbMKFz6FAsUtgAab', 'New Releases'),
 ('0JQ5DAqbMKFIcJ8h2w86fL', "New Year's"),
 ('0JQ5DAqbMKFEC4WFtoNRpw', 'Pop'),
 ('0JQ5DAqbMKFQ00XGBls6ym', 'Hip-Hop'),
 ('0JQ5DAqbMKFzHmL4tf05da', 'Mood'),
 ('0JQ5DAudkNjCgYMM0TZXDw', 'Charts'),
 ('0JQ5DAqbMKFDXXwE9BDJAr', 'Rock'),
 ('0JQ5DAqbMKFxXaXKP7zcDp', 'Latin'),
 ('0JQ5DAqbMKFHOzuVTgTizF', 'Dance/Electronic'),
 ('0JQ5IMCbQBLkUxOEesHoGn', 'Skate Noise'),
 ('0JQ5DAtOnAEpjOgUKwXyxj', 'Discover'),
 ('0JQ5DAqbMKFCWjUTdzaG0e', 'Indie'),
 ('0JQ5DAqbMKFImHYGo3eTSg', 'Fresh Finds'),
 ('0JQ5DAqbMKFPw634sFwguI', 'EQUAL'),
 ('0JQ5DAqbMKFOOxftoKZxod', 'RADAR'),
 ('0JQ5DAqbMKFylzQHdDVGXB', 'Schlager'),
 ('0JQ5DAqbMKFAXlCG6QvYQ4', 'Workout'),
 ('0JQ5DAqbMKFEZPnFQSFB1T', 'R&B'),
 ('0JQ5DAqbMKFGvOw3O4nLAf', 'K-pop'),
 ('0JQ5DAqbMKFFzDl7qN9Apr', 'Chill'),
 ('0JQ5DAqbMKFCuoRTxhYWow', 'Sleep'),
 ('0JQ5DAqbMKFA6SOHvT3gck', 'Party'),
 ('0JQ5DAqbMKFx0uLQR2okcc', 'At H

In [ ]:
preferred_genres = ['afrobeat', 'alt-rock', 'alternative','bluegrass', 'blues', 'bossanova', 
                    'dancehall', 'deep-house', 'disco', 'drum-and-bass', 'dub', 'dubstep', 
                    'funk', 'groove', 'hard-rock', 'house', 'indie', 'indie-pop', 'jazz', 'punk-rock', 
                    'reggae', 'rock', 'rock-n-roll', 'rockabilly', 'ska', 'soul', 'trip-hop']

In [ ]:
# target_bpm = 110.0
# bpm_bandwidth = 10.0
min_tempo = 120.0
max_tempo = 130.0
query = {
    'seed_genres': ['jazz'],
    'limit': 20,
    'country': 'CH',
}
track_attributes = {
#     'min_tempo': target_bpm - (bpm_bandwidth / 2),
#     'max_tempo': target_bpm + (bpm_bandwidth / 2),
    'min_tempo': min_tempo,
    'max_tempo': max_tempo,
    'min_danceability': 0.66,
    'max_liveness': 0.33,
    'max_speechiness': 0.33,
    # 'max_instrumentalness': 0.33,
    'max_acousticness': 0.33,
    'target_time_signature': 4,
    # 'min_valence': 0.5,
    'min_popularity': 1,
}
response = sp.recommendations(**query, **track_attributes)
tracks = response['tracks']
len(tracks)

In [ ]:
response.keys()

In [ ]:
response['seeds']

In [ ]:
tracklist = []
for track in tracks:
    tracklist.append({
        'id': track['id'],
        'name': track['name'],
        'artist_id': track['artists'][0]['id'],
        'artist_name': track['artists'][0]['name'],
        'popularity': track['popularity'],
    })
df = pd.DataFrame(tracklist)

In [ ]:
df

In [ ]:
df

In [ ]:
df_sorted = df.sort_values('tempo')
df['cum_mins'] = (df_sorted['duration_ms'].cumsum()) / 1000 / 60
df[['id', 'artist_name', 'name', 'tempo', 'duration_ms', 'cum_mins']]

In [ ]:
playlist_name = (str(min_tempo) + "-" + str(max_tempo) + " BPM " + ', '.join(query['seed_genres']).title())
playlist_name

In [ ]:
playlist = sp.user_playlist_create(
    user['id'], 
    playlist_name, 
    public=False, 
    description="Created with BPMPlaylist.py"
)
sp.user_playlist_add_tracks(user['id'], playlist['id'], list(df['id']))
sp.user_playlist(user['id'], playlist['id'], fields="tracks")['tracks']['total']